In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
# Function to parse each line of the file
def parse_line(line):
    # Split the line by commas for the titles or by spaces for the data
    parts = line.strip().split(',') if ',' in line else line.strip().split()
    return parts

# Specify the names of the files to search for
target_files = ['flow_cut_to_failure.vtk', 'transport_cut_to_failure.vtk']
n = 135

# Arrays to store the read data
h2aq = np.zeros([n+1, 1])
h2saq = np.zeros([n+1, 1])
h2g = np.zeros([n+1, 1])

In [17]:
for i in range(1,n+1):
    # Get the current directory
    current_dir = './../Hem_Base/run_' + str(i) + '/'

    # Search for the target files in the current directory
    for file in os.listdir(current_dir):
        if file in target_files:
            print("Failure found in folder:", current_dir)
            break  # Exit the loop if either file is found
        else:
            read_folder = True
            
    if not os.path.exists('./../Hem_Base/run_' + str(i) + '/pflotran_h2-012.vtk'):
            print(f"File does not exist: {'./../Hem_Base/run_' + str(i) + '/pflotran_h2-012.vtk'}")
            continue
    else:
        file_path_h2 = './../Hem_Base/run_' + str(i) + '/pflotran_h2-012.vtk'
        #============
        # Read H2(aq)
        # ============
        # Initialize an empty list to store the numbers
        numbers_list_h2 = []
        numbers_list_sat = []

        # Define a flag to identify when to start reading numbers
        start_reading_h2 = False
        start_reading_sat = False

        # Open the file and read line by line
        with open(file_path_h2, 'r') as file:
            for line in file:
                # Check if the line contains the title indicating the start of numbers
                if 'SCALARS Total_H2(aq)' in line:
                    start_reading_h2 = True
                    # Skip the next line which is 'LOOKUP_TABLE default'
                    next(file)
                elif start_reading_h2:
                    # If the line is empty or a new section starts, stop reading
                    if line.strip() == '' or 'SCALARS' in line or 'POINT_DATA' in line:
                        start_reading_h2 = False
                    else:
                        # Split the line into individual numbers and extend the list
                        numbers_list_h2.extend([float(num) for num in line.split()])

                if 'SCALARS Liquid_Saturation' in line:
                    start_reading_sat = True
                    # Skip the next line which is 'LOOKUP_TABLE default'
                    next(file)
                elif start_reading_sat:
                    # If the line is empty or a new section starts, stop reading
                    if line.strip() == '' or 'SCALARS' in line or 'POINT_DATA' in line:
                        start_reading_sat = False
                    else:
                        # Split the line into individual numbers and extend the list
                        numbers_list_sat.extend([float(num) for num in line.split()])

        # Convert the list to a numpy array
        numbers_array_h2_all = np.array(numbers_list_h2)
        numbers_array_h2 = numbers_array_h2_all[numbers_array_h2_all <= 1.0]
        numbers_array_sat_all = np.array(numbers_list_sat)
        numbers_array_sat = numbers_array_sat_all[numbers_array_h2_all <= 1.0]

        cell_vol_gr = 16*16*20*0.2*1000

        # Display the first few numbers to verify
        cell_h2 = cell_vol_gr*numbers_array_sat*numbers_array_h2
        total_h2 = np.sum(cell_h2)

        # print("Total H2(aq) is " + str(total_h2) + " moles")
        h2aq[i, 0] = total_h2

        # ============
        # Read H2S(aq)
         # ============
        file_path_h2s = './../Hem_Base/run_' + str(i) + '/pflotran_h2-012.vtk'
        # Initialize an empty list to store the numbers
        numbers_list_h2s = []
        numbers_list_sat = []

        # Define a flag to identify when to start reading numbers
        start_reading_h2s = False
        start_reading_sat = False

        # Open the file and read line by line
        with open(file_path_h2s, 'r') as file:
            for line in file:
                # Check if the line contains the title indicating the start of numbers
                if 'SCALARS H2S(aq)' in line:
                    start_reading_h2s = True
                    # Skip the next line which is 'LOOKUP_TABLE default'
                    next(file)
                elif start_reading_h2s:
                    # If the line is empty or a new section starts, stop reading
                    if line.strip() == '' or 'SCALARS' in line or 'POINT_DATA' in line:
                        start_reading_h2s = False
                    else:
                        # Split the line into individual numbers and extend the list
                        numbers_list_h2s.extend([float(num) for num in line.split()])

                if 'SCALARS Liquid_Saturation' in line:
                    start_reading_sat = True
                    # Skip the next line which is 'LOOKUP_TABLE default'
                    next(file)
                elif start_reading_sat:
                    # If the line is empty or a new section starts, stop reading
                    if line.strip() == '' or 'SCALARS' in line or 'POINT_DATA' in line:
                        start_reading_sat = False
                    else:
                        # Split the line into individual numbers and extend the list
                        numbers_list_sat.extend([float(num) for num in line.split()])

        # Convert the list to a numpy array
        numbers_array_h2s_all = np.array(numbers_list_h2s)
        numbers_array_h2s = numbers_array_h2s_all[(numbers_array_h2s_all <= 1.0) & (numbers_array_h2s_all >= 2e-8)]
        # numbers_array_sat_all = np.array(numbers_list_sat)
        # numbers_array_sat = numbers_array_sat_all[numbers_array_h2s_all <= 1.0 and numbers_array_h2s_all >= 2e-8]

        # cell_vol_gr = 16*16*20*0.2*1000

        # # Display the first few numbers to verify
        # cell_h2s = cell_vol_gr*numbers_array_sat*numbers_array_h2s
        # total_h2s = np.sum(cell_h2s)

        # print("Total H2S(aq) is: " + str(total_h2s) + " moles")
        h2saq[i, 0] = np.average(numbers_array_h2s) # total_h2s

        # ==========
        # Read H2(g)
        # ==========
        # Define the path to your .dat file
        file_path_h2g = './../Hem_Base/run_' + str(i) + '/pflotran_h2-mas.dat'

        # Read the file and parse each line
        with open(file_path_h2g, 'r') as file:
            lines = file.readlines()

        # Parse the column titles
        column_titles = parse_line(lines[0])

        # Initialize a list to hold the parsed data
        data = []

        # Parse each line of data
        for line in lines[1:]:  # Skip the first line with column titles
            parsed_line = parse_line(line)
            if parsed_line:  # Ensure the line is not empty
                data.append(parsed_line)

        # Create a DataFrame from the parsed data
        # The length of the longest row is used to ensure all rows have the same length
        max_length = max(len(row) for row in data)
        df = pd.DataFrame(data, columns=column_titles[:max_length])

        df.columns = df.columns.str.strip('"')
        last_number_global_h2 = df["Global H2(g) [mol]"].iloc[11]
        last_number_global_h2 = pd.to_numeric(last_number_global_h2, errors='coerce')
        h2g[i, 0] = last_number_global_h2
        print("The H2(g) [mol] after 1Y is: ", last_number_global_h2)


The H2(g) [mol] after 1Y is:  90566863300.0
The H2(g) [mol] after 1Y is:  90478472600.0
The H2(g) [mol] after 1Y is:  90315436200.0
The H2(g) [mol] after 1Y is:  90171883600.0
The H2(g) [mol] after 1Y is:  90042886900.0
The H2(g) [mol] after 1Y is:  89873719500.0
The H2(g) [mol] after 1Y is:  89504067000.0
The H2(g) [mol] after 1Y is:  89325608100.0
The H2(g) [mol] after 1Y is:  89225231200.0
The H2(g) [mol] after 1Y is:  89160306500.0
The H2(g) [mol] after 1Y is:  89114489700.0
The H2(g) [mol] after 1Y is:  89080253400.0
The H2(g) [mol] after 1Y is:  89053615600.0
The H2(g) [mol] after 1Y is:  89032255700.0
The H2(g) [mol] after 1Y is:  89014722600.0
The H2(g) [mol] after 1Y is:  90566587600.0
The H2(g) [mol] after 1Y is:  90308461600.0
The H2(g) [mol] after 1Y is:  89898742600.0
The H2(g) [mol] after 1Y is:  89608747800.0
The H2(g) [mol] after 1Y is:  89426417900.0
The H2(g) [mol] after 1Y is:  89266271100.0
The H2(g) [mol] after 1Y is:  89043953600.0
The H2(g) [mol] after 1Y is:  88

In [18]:
print(h2aq)
print(h2saq)
print(h2g)

np.savetxt('./H2AQ_hem_Base_1Y.txt', h2aq, fmt='%d', delimiter=', ')
np.savetxt('./H2SAQ_hem_Base_1Y.txt', h2saq, fmt='%.10e', delimiter=', ')
np.savetxt('./H2G_hem_Base_1Y.txt', h2g, fmt='%d', delimiter=', ')

[[0.00000000e+00]
 [7.57613670e+08]
 [7.56887608e+08]
 [7.55519931e+08]
 [7.54313824e+08]
 [7.53229608e+08]
 [7.51807593e+08]
 [7.48700539e+08]
 [7.47200663e+08]
 [7.46356980e+08]
 [7.45811217e+08]
 [7.45426038e+08]
 [7.45138182e+08]
 [7.44914195e+08]
 [7.44734573e+08]
 [7.44587123e+08]
 [7.57631396e+08]
 [7.55472962e+08]
 [7.52029283e+08]
 [7.49591532e+08]
 [7.48058940e+08]
 [7.46712735e+08]
 [7.44843130e+08]
 [7.44090566e+08]
 [7.43673131e+08]
 [7.43405458e+08]
 [7.43218559e+08]
 [7.43080340e+08]
 [7.42973870e+08]
 [7.42889273e+08]
 [7.42820402e+08]
 [7.57643867e+08]
 [7.45129905e+08]
 [7.49486251e+08]
 [7.47248204e+08]
 [7.46101629e+08]
 [7.45129905e+08]
 [7.43783887e+08]
 [7.43250551e+08]
 [7.42959866e+08]
 [7.42775942e+08]
 [7.42648754e+08]
 [7.42555398e+08]
 [7.42483879e+08]
 [7.42427294e+08]
 [7.42381366e+08]
 [7.53025855e+08]
 [7.52807901e+08]
 [7.52369669e+08]
 [7.51983966e+08]
 [7.51636683e+08]
 [7.51178414e+08]
 [7.50162370e+08]
 [7.49646785e+08]
 [7.49335803e+08]
 [7.491221